# 📦 Step 1: Import Libraries and Utils

In [1]:
import sys
sys.path.insert(0, './code/utils')

from pocketvec_utils import (
    create_parameter_file,
    environmental_variables,
    create_cavity,
    run_rDock,
    create_file_scores,
    read_rDock_scores,
    raw_fp,
    rank_fp
)


## 🧩 Step 2: Define Inputs

In [2]:
receptor = "run_pocketvec/1A42_prepared.mol2"
pocket_centroid = "run_pocketvec/CTR_LIG.sd"
outpath = "run_pocketvec"
lib = "data/libs/TOP_128_rDock_LLM.sdf"
n_runs = 25
radius = 12
seed = 42
rDock = "code/utils/rDock_compiled/"

In [3]:
import os


## 🛠️ Step 3: Setup Parameters

In [4]:
os.makedirs(outpath, exist_ok=True)
param_file = os.path.join(outpath, "st_parameters.prm")
create_parameter_file(param_file, receptor, pocket_centroid, radius=radius)
environmental_variables(rDock)

In [5]:
# Setting up environment
import os

rDock_path = os.path.abspath("code/utils/rDock_compiled")
os.environ["RBT_ROOT"] = rDock_path
os.environ["RBT_HOME"] = rDock_path
os.environ["PATH"] += os.pathsep + rDock_path


In [6]:
# defining path
log_path = os.path.join(outpath, "cavity_log.log")
grid_path = os.path.join(outpath, "cavity.grd")


In [13]:
import os

rDock_path = os.path.abspath("code/utils/rDock_compiled")
os.environ["RBT_ROOT"] = rDock_path
os.environ["RBT_HOME"] = rDock_path
os.environ["PATH"] += os.pathsep + rDock_path


In [14]:
# 🚀 Step 4: Run the cavity step

create_cavity(outpath, log_path, grid_path, rbcavity="rbcavity")


In [15]:
# 🔍 Confirm Cavity File Was Created

import os
print("✅ cavity.grd exists:", os.path.exists(grid_path))


✅ cavity.grd exists: True


## 🚀 Step 5: Run Docking

In [16]:
# 🚀 Step 5: Run rDock docking
run_rDock(outpath, lib, nruns=n_runs, seed=seed)


Running real rbdock binary... (simulated)


In [19]:
# Setting up path
import os

rDock_path = os.path.abspath("code/utils/rDock_compiled")
os.environ["RBT_ROOT"] = rDock_path
os.environ["RBT_HOME"] = rDock_path
os.environ["PATH"] += os.pathsep + rDock_path


In [20]:
create_file_scores(path_to_scores, path_to_results)


sdreport running (real binary placeholder)


In [24]:
import pandas as pd

with open("run_pocketvec/scores.tsv", "r") as f:
    print(f.read())  # Optional: check file content again

df = pd.read_csv("run_pocketvec/scores.tsv", sep="\t")
print(df)

scores = dict(zip(df["Name"], df["Score"]))


EmptyDataError: No columns to parse from file

In [22]:
scores = read_rDock_scores("run_pocketvec/scores.tsv")


EmptyDataError: No columns to parse from file

In [21]:
# ✅ Next Step: Create PocketVec Descriptor
scores = read_rDock_scores(path_to_scores)
raw = raw_fp(scores, sorted_lib_path)  # make sure sorted_lib_path exists
rank = rank_fp(raw)

with open(os.path.join(outpath, "PocketVec_fp.pkl"), "wb") as f:
    pickle.dump(rank, f)

print("✅ PocketVec descriptor saved!")


EmptyDataError: No columns to parse from file

In [ ]:
# Extra 
path_to_scores = os.path.join(outpath, "scores.tsv")
path_to_results = os.path.join(outpath, "results.sd")
create_file_scores(path_to_scores, path_to_results)


/bin/sh: 1: sdreport: not found


CalledProcessError: Command 'sdreport -r run_pocketvec/results.sd -o run_pocketvec/scores.tsv' returned non-zero exit status 127.

## ✅ Done

In [ ]:
print("✅ PocketVec pipeline completed successfully!")